In [1]:
import json
import random
import numpy as np
import pandas as pd
from datasets import load_from_disk, load_dataset
from datasets import Dataset
from tqdm import tqdm, trange
import kss

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
datasets = load_from_disk("../../data/train_dataset")

In [3]:
datasets['train'][0]

{'title': '미국 상원',
 'context': '미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국 의회의 상원이다.\\n\\n미국 부통령이 상원의장이 된다. 각 주당 2명의 상원의원이 선출되어 100명의 상원의원으로 구성되어 있다. 임기는 6년이며, 2년마다 50개주 중 1/3씩 상원의원을 새로 선출하여 연방에 보낸다.\\n\\n미국 상원은 미국 하원과는 다르게 미국 대통령을 수반으로 하는 미국 연방 행정부에 각종 동의를 하는 기관이다. 하원이 세금과 경제에 대한 권한, 대통령을 포함한 대다수의 공무원을 파면할 권한을 갖고 있는 국민을 대표하는 기관인 반면 상원은 미국의 주를 대표한다. 즉 캘리포니아주, 일리노이주 같이 주 정부와 주 의회를 대표하는 기관이다. 그로 인하여 군대의 파병, 관료의 임명에 대한 동의, 외국 조약에 대한 승인 등 신속을 요하는 권한은 모두 상원에게만 있다. 그리고 하원에 대한 견제 역할(하원의 법안을 거부할 권한 등)을 담당한다. 2년의 임기로 인하여 급진적일 수밖에 없는 하원은 지나치게 급진적인 법안을 만들기 쉽다. 대표적인 예로 건강보험 개혁 당시 하원이 미국 연방 행정부에게 퍼블릭 옵션(공공건강보험기관)의 조항이 있는 반면 상원의 경우 하원안이 지나치게 세금이 많이 든다는 이유로 퍼블릭 옵션 조항을 제외하고 비영리건강보험기관이나 보험회사가 담당하도록 한 것이다. 이 경우처럼 상원은 하원이나 내각책임제가 빠지기 쉬운 국가들의 국회처럼 걸핏하면 발생하는 의회의 비정상적인 사태를 방지하는 기관이다. 상원은 급박한 처리사항의 경우가 아니면 법안을 먼저 내는 경우가 드물고 하원이 만든 법안을 수정하여 다시 하원에 되돌려보낸다. 이러한 방식으로 단원제가 빠지기 쉬운 함정을 미리 방지하는 것이다.날짜=2017-02-05',
 'question': '대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?',
 'id': 'mrc-1-000067',
 'answers': {'answer_start'

In [23]:
new = []
for i in range(len(datasets['train'])):
    new.append({
            'id': datasets['train'][i]['id'],
            'question': datasets['train'][i]['question'],
            'context': datasets['train'][i]['title'] + '. ' + datasets['train'][i]['context'],
            'answers': {
                'text': datasets['train'][i]['answers']['text'],
                'answer_start': [int(datasets['train'][i]['answers']['answer_start'][0]) + len(datasets['train'][i]['title']) + 2]
            }
        })
new[0]

{'id': 'mrc-1-000067',
 'question': '대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?',
 'context': '미국 상원. 미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국 의회의 상원이다.\\n\\n미국 부통령이 상원의장이 된다. 각 주당 2명의 상원의원이 선출되어 100명의 상원의원으로 구성되어 있다. 임기는 6년이며, 2년마다 50개주 중 1/3씩 상원의원을 새로 선출하여 연방에 보낸다.\\n\\n미국 상원은 미국 하원과는 다르게 미국 대통령을 수반으로 하는 미국 연방 행정부에 각종 동의를 하는 기관이다. 하원이 세금과 경제에 대한 권한, 대통령을 포함한 대다수의 공무원을 파면할 권한을 갖고 있는 국민을 대표하는 기관인 반면 상원은 미국의 주를 대표한다. 즉 캘리포니아주, 일리노이주 같이 주 정부와 주 의회를 대표하는 기관이다. 그로 인하여 군대의 파병, 관료의 임명에 대한 동의, 외국 조약에 대한 승인 등 신속을 요하는 권한은 모두 상원에게만 있다. 그리고 하원에 대한 견제 역할(하원의 법안을 거부할 권한 등)을 담당한다. 2년의 임기로 인하여 급진적일 수밖에 없는 하원은 지나치게 급진적인 법안을 만들기 쉽다. 대표적인 예로 건강보험 개혁 당시 하원이 미국 연방 행정부에게 퍼블릭 옵션(공공건강보험기관)의 조항이 있는 반면 상원의 경우 하원안이 지나치게 세금이 많이 든다는 이유로 퍼블릭 옵션 조항을 제외하고 비영리건강보험기관이나 보험회사가 담당하도록 한 것이다. 이 경우처럼 상원은 하원이나 내각책임제가 빠지기 쉬운 국가들의 국회처럼 걸핏하면 발생하는 의회의 비정상적인 사태를 방지하는 기관이다. 상원은 급박한 처리사항의 경우가 아니면 법안을 먼저 내는 경우가 드물고 하원이 만든 법안을 수정하여 다시 하원에 되돌려보낸다. 이러한 방식으로 단원제가 빠지기 쉬운 함정을 미리 방지하는 것이다.날짜=2017-02-05',
 'answers': {'text': ['하원'], 'answer_st

In [22]:
new[230]['context'][144]

'영'

In [28]:
title_df = pd.DataFrame(data=new)
title_df.head()

,id,question,context,answers
0,mrc-1-000067,대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?,미국 상원. 미국 상의원 또는 미국 상원(United States Senate)은 ...,"{'text': ['하원'], 'answer_start': [242]}"
1,mrc-0-004397,현대적 인사조직관리의 시발점이 된 책은?,인사조직관리. '근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되...,"{'text': ['《경영의 실제》'], 'answer_start': [220]}"
2,mrc-1-000362,강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?,강희제. 강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에...,"{'text': ['백성'], 'answer_start': [515]}"
3,mrc-0-001510,11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?,"금동삼존불감. 불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작...","{'text': ['중국'], 'answer_start': [633]}"
4,mrc-0-000823,명문이 적힌 유물을 구성하는 그릇의 총 개수는?,계사명 사리구. 동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로...,"{'text': ['4개'], 'answer_start': [39]}"


In [29]:
Dataset.from_pandas(title_df).save_to_disk(f"../../add_title/train_dataset/train")

In [30]:
valid = []
for i in range(len(datasets['validation'])):
    valid.append({
            'id': datasets['validation'][i]['id'],
            'question': datasets['validation'][i]['question'],
            'context': datasets['validation'][i]['context'],
            'answers': {
                'text': datasets['validation'][i]['answers']['text'],
                'answer_start': datasets['validation'][i]['answers']['answer_start']
            }
        })
len(valid)

240

In [31]:
valid_df = pd.DataFrame(data=valid)
valid_df.head()

,id,question,context,answers
0,mrc-0-003264,처음으로 부실 경영인에 대한 보상 선고를 받은 회사는?,"순천여자고등학교 졸업, 1973년 이화여자대학교를 졸업하고 1975년 제17회 사법...","{'text': ['한보철강'], 'answer_start': [284]}"
1,mrc-0-004762,스카버러 남쪽과 코보콘그 마을의 철도 노선이 처음 연장된 연도는?,요크 카운티 동쪽에 처음으로 여객 열차 운행이 시작한 시점은 1868년 토론토 & ...,"{'text': ['1871년'], 'answer_start': [146]}"
2,mrc-1-001810,촌락에서 운영 위원 후보자 이름을 쓰기위해 사용된 것은?,"촐라 정부\n 촐라의 정부 체제는 전제군주제였으며,2001 촐라의 군주는 절대적인 ...","{'text': ['나뭇잎'], 'answer_start': [517]}"
3,mrc-1-000219,로타이르가 백조를 구하기 위해 사용한 것은?,프랑스의 십자군 무훈시는 1099년 예루살렘 왕국의 통치자가 된 고드프루아 드 부용...,"{'text': ['금대야'], 'answer_start': [1109]}"
4,mrc-1-000285,의견을 자유롭게 나누는 것은 조직 내 어떤 관계에서 가능한가?,탈관료제화는 현대사회에서 관료제 성격이 약화되는 현상이다. 현대사회에서 관료제는 약...,"{'text': ['수평적 관계'], 'answer_start': [386]}"


In [32]:
Dataset.from_pandas(valid_df).save_to_disk(f"../../add_title/train_dataset/validation")

In [13]:
with open('../../data/train_dataset/dataset_dict.json') as f:
    json_data = json.load(f)
with open('../../add_title/train_dataset/dataset_dict.json', 'w') as f:
    json.dump(json_data, f)

In [14]:
with open('../../data/wikipedia_documents.json') as f:
    wiki_data = json.load(f)
with open('../../concat_shuffle/wikipedia_documents.json', 'w') as f:
    json.dump(wiki_data, f)